In [11]:
import zipfile
import os

zip_path = "archive.zip"

extract_dir = "att_faces"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Dataset extracted to: {os.path.abspath(extract_dir)}")

Dataset extracted to: /drive/notebooks/att_faces


In [12]:
from PIL import Image
import numpy as np
from scipy.stats import multivariate_normal

data_matrix = np.zeros((400, 10304))
row_index = 0

dataset_path = "att_faces"

for subject in range(1, 41):
    subject_dir = os.path.join(dataset_path, f"s{subject}")

    for img_num in range(1, 11):
        img_path = os.path.join(subject_dir, f"{img_num}.pgm")

        img = Image.open(img_path)
        img_array = np.array(img).flatten()

        data_matrix[row_index, :] = img_array
        row_index += 1

print(data_matrix.shape)

(400, 10304)


In [13]:
y = np.zeros(400, dtype=int)

for i in range (40):
    y[i * 10 : (i + 1) * 10] = i + 1

In [14]:
X_train = data_matrix[::2]
y_train = y[::2]

X_test = data_matrix[1::2]
y_test = y[1::2]

In [15]:
import os
for f in ['eigenvalues.npy', 'eigenvectors.npy']:
    if os.path.exists(f):
        os.remove(f)


In [16]:
class PCA:
    def __init__(self, n_components=None, alpha=None):
        """
        Parameters:
        - n_components: int, number of components to keep
        - alpha: float (0-1), minimum variance ratio to retain
        """
        self.n_components = n_components
        self.alpha = alpha
        self.components = None
        self.mean = None
        self.explained_variance_ratio = None
    
    def fit(self, X):
        # 1. Standardize the data (center to mean)
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean
        
        # Save/load eigenvalues and eigenvectors
        if os.path.exists('eigenvalues.npy') and os.path.exists('eigenvectors.npy'):
            self.eigenvalues = np.load('eigenvalues.npy')
            self.eigenvectors = np.load('eigenvectors.npy')
        else:
            cov_matrix = np.cov(X_centered, rowvar=False)
            eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
            sorted_indices = np.argsort(eigenvalues)[::-1]
            self.eigenvalues = eigenvalues[sorted_indices]
            self.eigenvectors = eigenvectors[:, sorted_indices]
            
            np.save('eigenvalues.npy', self.eigenvalues)
            np.save('eigenvectors.npy', self.eigenvectors)
        
        # 5. Compute explained variance ratio ??
        total_variance = np.sum(self.eigenvalues)
        self.explained_variance_ratio = self.eigenvalues / total_variance
        
        # 6. Determine number of components to keep 
        if self.alpha is not None:
            cumulative_variance = np.cumsum(self.explained_variance_ratio)
            self.n_components = np.argmax(cumulative_variance >= self.alpha) + 1
        
        # 7. Select top components with max eigenvalue
        self.components = self.eigenvectors[:, :self.n_components] ## using in inverse transform
        
        return self
    
    def transform(self, X):
        # Center the data using the mean from training
        X_centered = X - self.mean
        
        # Project data onto principal components
        return np.dot(X_centered, self.components)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

In [17]:
def apply_pca(X_train, X_test, alpha=0.9): ## default with 99%
    """"
    make a pojection for x_test  not need to fit but train must fit then make a projuction
    make a fit transform for x_train 

    """
    pca = PCA(alpha=alpha)  
    X_train_pca = pca.fit_transform(X_train)  
    X_test_pca = pca.transform(X_test) 
    return X_train_pca, X_test_pca, pca

# Apply PCA with different thresholds
alphas = [0.8, 0.85, 0.9, 0.95] ## the information needed to keep from pca taining 
pca_results = {}

for alpha in alphas:
    X_train_pca, X_test_pca, pca = apply_pca(X_train, X_test, alpha)
    pca_results[alpha] = {
        'train': X_train_pca,
        'test': X_test_pca,
        'pca': pca,
        'n_components': pca.n_components,  # Fixed: no underscore
        'explained_variance': pca.explained_variance_ratio  # Fixed: no underscore
    }
    print(f"Alpha: {alpha}, Components: {pca.n_components}, Explained Variance: {np.sum(pca.explained_variance_ratio):.4f}")

<class 'numpy._core._exceptions._ArrayMemoryError'>: Unable to allocate 810. MiB for an array with shape (10304, 10304) and data type float64

In [27]:
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
import numpy as np
from scipy.special import logsumexp
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


class GMM:
    def __init__(self, K):
        self.K = K

    def initialize_params(self):
        self.r = np.full((self.N, self.K), 1 / self.K)
        self.pi = np.full(self.K, 1 / self.K)

        kmeans = KMeans(n_clusters=self.K, n_init=10, random_state=0).fit(self.data)
        self.mean = kmeans.cluster_centers_

        self.cov = np.array([np.eye(self.M) for _ in range(self.K)])

    def expectation(self):
        log_r = np.zeros((self.N, self.K))

        for k in range(self.K):
            rv = multivariate_normal(mean=self.mean[k], cov=self.cov[k], allow_singular=True)
            log_r[:, k] = np.log(self.pi[k] + 1e-10) + rv.logpdf(self.data)

       
        logsumexp_r = logsumexp(log_r, axis=1, keepdims=True)
        self.r = np.exp(log_r - logsumexp_r)

    def update_mixing(self):
        self.pi = np.sum(self.r, axis=0) / self.N

    def update_mean(self):
        for k in range(self.K):
            weight_sum = np.sum(self.r[:, k])
            self.mean[k] = np.sum(self.r[:, k][:, None] * self.data, axis=0) / weight_sum

    def update_cov(self):
        for k in range(self.K):
            cov_k = np.zeros((self.M, self.M))
            for n in range(self.N):
                diff = self.data[n] - self.mean[k]
                cov_k += self.r[n, k] * np.outer(diff, diff)
            self.cov[k] = cov_k / np.sum(self.r[:, k]) + 1e-6 * np.eye(self.M)

    def maximization(self):
        self.update_mixing()
        self.update_mean()
        self.update_cov()

    def log_likelihood(self):
        log_r = np.zeros((self.N, self.K))
        for k in range(self.K):
            rv = multivariate_normal(mean=self.mean[k], cov=self.cov[k], allow_singular=True)
            log_r[:, k] = np.log(self.pi[k] + 1e-10) + rv.logpdf(self.data)
        return np.sum(logsumexp(log_r, axis=1))

    def fit(self, data, max_iters=100, tol=1e-5):
        self.data = data
        self.N, self.M = data.shape
        self.initialize_params()

        prev_log_likelihood = None
        for iteration in range(max_iters):
            self.expectation()
            self.maximization()

            log_likelihood = self.log_likelihood()

            if prev_log_likelihood is not None and abs(log_likelihood - prev_log_likelihood) < tol:
                break
            prev_log_likelihood = log_likelihood

    def predict_proba(self, x):
        log_probs = np.array([
            np.log(self.pi[k] + 1e-10) + multivariate_normal(mean=self.mean[k], cov=self.cov[k], allow_singular=True).logpdf(x)
            for k in range(self.K)
        ])
        log_probs -= logsumexp(log_probs)
        return np.exp(log_probs)

    def predict(self, test_data):
        return np.array([np.argmax(self.predict_proba(x)) for x in test_data])

    def assign_clusters_to_classes(self, y_train):
        self.cluster_label_map = {}
        for k in range(self.K):
            scores = np.zeros(40)
            for n in range(self.N):
                class_label = y_train[n]
                scores[class_label - 1] += self.r[n, k]
            self.cluster_label_map[k] = np.argmax(scores) + 1

    def evaluate_accuracy(self, X_test, y_test):
        predicted_clusters = g.predict(X_test)
        y_pred = np.array([g.cluster_label_map.get(c, -1) for c in predicted_clusters])
        
        accuracy = accuracy_score(y_test, y_pred)

        f1 = f1_score(y_test, y_pred, average='macro')

        cm = confusion_matrix(y_test, y_pred)

        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1-score : {f1:.4f}")
        print("Confusion matrix:")
        print(cm)
        return accuracy, f1, cm


In [29]:
from sklearn.preprocessing import StandardScaler

alphas =[0.8, 0.85, 0.9, 0.95]
K = [20, 40, 60]

for k in K:
    for alpha in alphas:
        # scaler = StandardScaler()
        # X_train_scaled = scaler.fit_transform(X_train)
        # X_test_scaled = scaler.transform(X_test)
        
        pca = PCA(alpha=alpha)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        g = GMM(k)
        g.fit(X_train_pca)
        g.assign_clusters_to_classes(y_train)
        
        print(f"Gmm accuracy with k = {k} and alpha = {alpha}: ")
        g.evaluate_accuracy(X_test_pca, y_test)
        

<class 'EOFError'>: No data left in file